In [1]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1007. Sputum Proteomics/P1007.3. Analyses/P1007.3.2. Distribution Tests/Output'
cur_date = "091424"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)

#reading in file
demographics_df = data.frame(read_excel("Input/SubjectInfo_091324.xlsx", sheet = 2))
proteomics_df = data.frame(read_excel("Input/Processed_Data_091324.xlsx", sheet = 2))
proteomics_df[,6:65] = apply(proteomics_df[,6:65], 2, as.numeric)

head(demographics_df)
head(proteomics_df)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Warning message in apply(proteomics_df[, 6:65], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(proteomics_df[, 6:65], 2, as.numeric):
“NAs introduced by coercion”
Warning message in apply(proteomics_df[, 6:65], 2, as.numeric):
“NAs introduced by coercion”


,Original_Subject_Number,Proteomic_Core_Subject_Number,Subject_Number,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Asthma_Status,Responder_Status,Pre_Sputum_Neutrophilia,Post_Sputum_Neutrophilia
,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
1,1,1,1,NR_M_1,M,B,NH,37,26.0,NAS,NR,77.2,74.489999999999995
2,6,3,2,NR_F_2,F,W,NH,26,21.9,NAS,NR,22.6,0
3,7,5,3,R_M_3,M,W,NH,40,29.8,NAS,R,13.26,41.85
4,9,7,4,R_M_4,M,W,NH,25,18.8,NAS,R,50.3,76.41
5,13,9,5,R_F_5,F,B,NH,34,34.5,NAS,R,20.23,37.96
6,15,11,6,R_M_6,M,As,NH,19,20.8,NAS,R,47.77,66.67


,Protein_Accession,Protein_Name,Gene_Name,Protein_Descriptions,Number_Unique_Peptides,NR_M_1.Post_1,NR_M_1.Pre_2,NR_F_2.Post_3,NR_F_2.Pre_4,R_M_3.Post_5,⋯,NR_F_26.Post_51,NR_F_26.Pre_52,NR_M_27.Post_53,NR_M_27.Pre_54,NR_M_28.Post_55,NR_M_28.Pre_56,NR_F_29.Post_57,NR_F_29.Pre_58,NR_M_30.Post_59,NR_M_30.Pre_60
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,P61626,LYSC_HUMAN,LYZ,Lysozyme C,21,68766900,77360460,97510540,61349770,69751540,⋯,NA,29564320,52770740,92673350,78747670,NA,52744850,62254460,126898500,74926060
2,P01877,IGHA2_HUMAN,IGHA2,Immunoglobulin heavy constant alpha 2,26,22186750,39625340,46473370,23963820,39971520,⋯,NA,21746510,20663020,34912700,26115250,NA,36867380,15831060,154372500,48185010
3,P01876,IGHA1_HUMAN,IGHA1,Immunoglobulin heavy constant alpha 1,19,15777140,33683860,33518360,24035330,31405400,⋯,NA,18806690,16857290,28826750,17092660,NA,27664970,14284090,63727370,26176050
4,P02768,ALBU_HUMAN,ALB,Albumin,111,28212910,15515600,13395080,19225470,41915780,⋯,NA,19471410,9366254,27893000,15780580,NA,22491090,18820820,22261030,34728060
5,P0DTE7;P0DTE8;P0DUB6,AMY1B_HUMAN;AMY1C_HUMAN;AMY1A_HUMAN,AMY1B;AMY1C;AMY1A,Alpha-amylase 1B;Alpha-amylase 1C;Alpha-amylase 1A,43,16069960,3754483,388746100,20953490,5696434,⋯,NA,11883850,5799354,25845120,33562940,NA,6053868,6425425,4049148,6781004
6,P01834,IGKC_HUMAN,IGKC,Immunoglobulin kappa constant,3,11402010,18704190,18756690,19384390,21055090,⋯,NA,11875440,9094482,9353938,8898546,NA,8115087,7418536,16145070,15644880


In [2]:
# creating a vector with the subject ids of the outliers
outlier_subjects = c("R_M_8", "NR_F_26", "NR_M_28")

# creating a variable that does the opposite of '%in%'
`%notin%` = Negate(`%in%`)

longer_proteomics_df = proteomics_df[,c(1,6:65)] %>%
    pivot_longer(cols = 2:61, names_to = "Subject_ID.Sample_ID", values_to = "Intensity") %>%   
    # just want a column that specifies the exposure separately
    separate(Subject_ID.Sample_ID, c("Subject_ID", "Sample_ID"), sep = "\\.") %>%  
    inner_join(demographics_df[,4:11]) %>%
    separate(Sample_ID, c("Time_Point", "Sample_Number"), sep = "_") %>%
    # filtering for subjects not in the outlier_subjects vector
    filter(Subject_ID %notin% outlier_subjects) 

head(longer_proteomics_df)

Joining with `by = join_by(Subject_ID)`


Protein_Accession,Subject_ID,Time_Point,Sample_Number,Intensity,Sex,Race,Ethnicity,Age,BMI,Asthma_Status,Responder_Status
<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
P61626,NR_M_1,Post,1,68766900,M,B,NH,37,26.0,NAS,NR
P61626,NR_M_1,Pre,2,77360460,M,B,NH,37,26.0,NAS,NR
P61626,NR_F_2,Post,3,97510540,F,W,NH,26,21.9,NAS,NR
P61626,NR_F_2,Pre,4,61349770,F,W,NH,26,21.9,NAS,NR
P61626,R_M_3,Post,5,69751540,M,W,NH,40,29.8,NAS,R
P61626,R_M_3,Pre,6,35046090,M,W,NH,40,29.8,NAS,R


In [3]:
# creating additional dataframes that are subsetted by sex and asthma status
sex_df = longer_proteomics_df %>%
    group_by(Sex) %>%
    group_split()

females_df = sex_df[[1]]
males_df = sex_df[[2]]

asthma_df = longer_proteomics_df %>%
    group_by(Asthma_Status) %>%
    group_split()

asthmatic_df = asthma_df[[1]]
nonasthmatic_df = asthma_df[[2]]

In [4]:
proteins = unique(longer_proteomics_df$Protein_Accession)

values_df = data.frame()
#create an empty dataframe for the loop function to place values into

timepoint_t_tests = function(dataframe, stratification){
    
    for (i in 1:length(proteins)){
        
         # pre df
        pre_df = dataframe %>%
            filter(Time_Point == "Pre", Protein_Accession == proteins[i])
        # post df
        post_df = dataframe %>%
            filter(Time_Point == "Post", Protein_Accession == proteins[i])
    
        # t test
        t_test = t.test(pre_df$Intensity, post_df$Intensity)
        
        # calculating log2FC to get directionality
        FC = log2(mean(pre_df$Intensity)/mean(post_df$Intensity))
    
        # contains stratification, protein, u stat, and p value
        values_vector = cbind(stratification, proteins[i], FC, t_test$statistic, t_test$p.value)
        values_df = rbind(values_df, values_vector)
    }    
    
    colnames(values_df) = c("Stratification", "Protein_Accession", "log2FC", "Statistic", 'P Value')
    values_df$`P Adj` <- p.adjust(values_df$`P Value`, method = "fdr")

    return(values_df)
}

# calling fn
timepoint_results_df = timepoint_t_tests(longer_proteomics_df, "None")
males_results_df = timepoint_t_tests(males_df, "Male")
females_results_df = timepoint_t_tests(females_df, "Female")
asthmatic_results_df = timepoint_t_tests(asthmatic_df, "Asthmatic")
nonasthmatic_results_df = timepoint_t_tests(nonasthmatic_df, "Nonasthmatic")

In [5]:
# creating 1 df
all_results_df = rbind(timepoint_results_df, males_results_df, females_results_df, asthmatic_results_df, nonasthmatic_results_df)
all_results_df[,3:5] = apply(all_results_df[,3:5], 2, as.numeric)

head(all_results_df)

,Stratification,Protein_Accession,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
t,None,P61626,-0.12479838,-0.6161562,0.5405340,0.9995849
t1,None,P01877,-0.37343439,-1.2121748,0.2311457,0.9995849
t2,None,P01876,-0.30109842,-1.2156616,0.2297277,0.9995849
t3,None,P02768,0.06815574,0.2806047,0.7801479,0.9995849
t4,None,P0DTE7;P0DTE8;P0DUB6,-1.03143492,-0.9768769,0.3361814,0.9995849
t5,None,P01834,-0.16397402,-0.8461956,0.4017529,0.9995849


In [6]:
all_results_df %>%
    filter(`P Value` < 0.05 & abs(log2FC) >= log2(1.5)) %>%
    group_by(Stratification) %>%
    count()

Stratification,n
<chr>,<int>
Asthmatic,7
Female,15
Male,6
Nonasthmatic,39
None,20


In [7]:
# exporting results
write.xlsx(all_results_df, paste0(Output,"/", "Proteomics_T_Test_Results_", cur_date, ".xlsx"), rowNames = FALSE)